### Triển khai trên Whisper

In [9]:
import locale
locale.getpreferredencoding = lambda : "UTF-8"
!pip install -q openai-whisper ffmpeg pydub

In [16]:
import whisper
model = whisper.load_model("medium", device="cuda")

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 113MiB/s]


In [31]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment

JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def record(sec=3):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio


def tts(sec=5):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('/content/audio.wav',"wb") as f:
    f.write(b)
  print("Finish recording. Send to model")
  result = model.transcribe("/content/audio.wav", language="vietnamese")
  return result["text"]

In [11]:
record()

Start recording...


<IPython.core.display.Javascript object>

In [32]:
tts()

Start recording...


<IPython.core.display.Javascript object>

Finish recording. Send to model


' Trông đầm gì đẹp bằng sen, lá xanh bông trắng lại trên nhị vàng.'

### PhoWhisper

In [20]:
!pip install -q transformers pydub

In [21]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large", device="cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/4.26k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [28]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment

JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def tts_pho(sec=5):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('/content/audio.wav',"wb") as f:
    f.write(b)
  print("Finish recording. Send to model")
  result = transcriber("/content/audio.wav")
  return result["text"]

In [29]:
tts_pho()

Start recording...


<IPython.core.display.Javascript object>

Finish recording. Send to model


'trong đầm dì đẹp bằng sen lá xanh bông trắng lại chen nhị vàng.'